# Introduction

In this notebook, I will implement a random forest classifier from scratch in Python and test it using scikit-learn's built-in random forest classifier and the red wine dataset. 


**Logic of Random Forest:**

The general idea of random forest is not very sophisticated. We are essentially ensembling a group of decision trees, each trained using a random subset of the data and features, and making a voting-based prediction. The reason we don't want all decision trees to be trained using the same data is because we want random variation between the trees, which may help the model to overcome noises in the data.

So My procedure in this notebook will be:
1. Implement the Decision Tree classifier
2. Implement Bootstrap and Random Subspace(used to randomly select data and features for each decision tree)
3. Implement the Random Forest Algorithm
4. Test my implementation with scikit-learn's random forest classifier

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline

import random
from pprint import pprint

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Decision Tree Functions

Random forest is based on decision tree. So before we begin our implementation let's first bring our decision tree implementation from last time here. Also, random forest algorithm aims to scatter each sub-tree as far as possible. So when training each decision tree, we will add a new parameter of random_subspace, which essentially randomly selects a subset of features that each tree can use to partition the data.

In [ ]:
import pandas as pd
import random


# 1. Train-Test-Split
def train_test_split(df, test_size):
    
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))

    indices = df.index.tolist()
    test_indices = random.sample(population=indices, k=test_size)

    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df


# 2. Distinguish categorical and continuous features
def determine_type_of_feature(df):
    
    feature_types = []
    n_unique_values_treshold = 15
    for feature in df.columns:
        if feature != "label":
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types


# 3. Accuracy
def calculate_accuracy(predictions, labels):
    predictions_correct = predictions == labels
    accuracy = predictions_correct.mean()
    
    return accuracy

In [ ]:
import numpy as np
import pandas as pd
import random


# 1. Decision Tree helper functions 
# (see "decision tree algorithm flow chart.png")

# 1.1 Data pure?
def check_purity(data):
    
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

    
# 1.2 Classify
def classify_data(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification


# 1.3 Potential splits?
def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):  # excluding the last column which is the label
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        potential_splits[column_index] = unique_values
    
    return potential_splits


# 1.4 Lowest Overall Entropy?
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy


def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy


def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)
            
            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value


# 1.5 Split data
def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "continuous":
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values >  split_value]
    
    # feature is categorical   
    else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values != split_value]
    
    return data_below, data_above

In [ ]:
def decision_tree_algorithm(df, counter=0, min_samples=2, max_depth=5, random_subspace=None):
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
    
    
    # base cases
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        
        return classification

    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data, random_subspace)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # check for empty data
        if len(data_below) == 0 or len(data_above) == 0:
            classification = classify_data(data)
            return classification
        
        # determine question
        feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "continuous":
            question = "{} <= {}".format(feature_name, split_value)
            
        # feature is categorical
        else:
            question = "{} = {}".format(feature_name, split_value)
        
        # instantiate sub-tree
        sub_tree = {question: []}
        
        # find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, max_depth, random_subspace)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, max_depth, random_subspace)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [ ]:
def predict_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    if comparison_operator == "<=":
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return predict_example(example, residual_tree)

    
# 3.2 All examples of the test data
def decision_tree_predictions(test_df, tree):
    predictions = test_df.apply(predict_example, args=(tree,), axis=1)
    return predictions

# Import Data

In [ ]:
full_data = pd.read_csv('/kaggle/input/red-wine-dataset/wineQualityReds.csv')
full_data['label'] = full_data['quality']
full_data = full_data.drop('quality', axis=1)

column_names = []
for column in full_data.columns:
    name = column.replace(" ", "_")
    column_names.append(name)
full_data.columns = column_names

full_data.head()

In [ ]:
def transform_label(value):
    if value <= 5:
        return "bad"
    else:
        return "good"

full_data["label"] = full_data.label.apply(transform_label)

In [ ]:
random.seed(0)
train_data, test_data = train_test_split(full_data, test_size=0.2)

# Bootstrap

Bootstrap is a key component of random forest algorithm. Since we want each decision tree to be different, we will not use the entire train set to train each decision tree. In contrast, we will randomly select a subset of data from the train set for each tree. This process of randomly selecting is bootstrapping. 

In [ ]:
def bootstrap(train_data, n_bootstrap):
    index = np.random.randint(low=0, high=len(train_data), size=n_bootstrap)
    boot_data = train_data.iloc[index]
    
    return boot_data

In [ ]:
bootstrap(train_data, n_bootstrap=100)

# Random Subspace

So like I previously mentioned, we don't want each decision tree to be trained using same data. So for each tree there will be a parameter to control which features can be used to partition the data.

In [ ]:
def get_potential_splits(data, random_subspace):
    
    potential_splits = {}
    _, n_columns = data.shape
    
    col_index = list(range(n_columns-1))
    if random_subspace and random_subspace <= len(col_index):
        col_index = random.sample(population=col_index, k=random_subspace)
    
    for column_index in col_index:  # excluding the last column which is the label
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        potential_splits[column_index] = unique_values
    
    return potential_splits

In [ ]:
get_potential_splits(train_data.values, random_subspace=3)

# Random Forest

Finally, we can write our random forest algorithm. The actual algorithm is actually not very sophisticated. For each tree in the forest, we are randomly picking a subset of train data and ask each decision tree to partition the data using random subspace. Then our forest is created!

In [ ]:
def  random_forest_algorithm(train_data, n_trees, n_bootstrap, n_features, max_depth):
    forest = []
    for i in range(n_trees):
        boot_data = bootstrap(train_data, n_bootstrap)
        tree = decision_tree_algorithm(boot_data, max_depth=max_depth, random_subspace=n_features)
        forest.append(tree)
        
    return forest

In [ ]:
forest = random_forest_algorithm(train_data, n_trees=4, n_bootstrap=800, n_features=4, max_depth=3)

In [ ]:
len(forest)

In [ ]:
pprint(forest[0])

In [ ]:
pprint(forest[1])

In [ ]:
pprint(forest[2])

In [ ]:
pprint(forest[3])

We can see how all the decision trees are different!

# Prediction

Now I am implementating a classifier, so our prediction will be voting-based, which means I will just use the mode of all results from all decision trees to construct my final prediction.

In [ ]:
def random_forest_prediction(test_data, forest):
    pred_data = {}
    for i in range(len(forest)):
        col = "tree...{}".format(i)
        predictions = decision_tree_predictions(test_data, tree=forest[i])
        pred_data[col] = predictions
    
    pred_data = pd.DataFrame(pred_data)
    
    return pred_data.mode(axis=1)[0]

In [ ]:
predictions = random_forest_prediction(test_data, forest)

accuracy = calculate_accuracy(predictions, test_data.label)
accuracy

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=4, criterion='entropy', max_depth=3, max_samples=800, random_state=21)
rf_model.fit(train_data.drop('label', axis=1), train_data['label'])

y_pred = rf_model.predict(test_data.drop('label', axis=1))
accuracy = calculate_accuracy(y_pred, test_data.label)
accuracy

So the final accuracy shows that our random forest classifier is pretty close to scikit-learn's random forest classifier, which would show the effectiveness of our implementation.